In [ ]:
print('Hello, world!')

In [ ]:
import pandas as pd
train_data = pd.read_csv('train.csv')
train_data.describe(include='all')

In [ ]:
transported_gender = train_data[train_data['Transported'] == 'True']['VIP'].value_counts(normalize=True) * 100
transported_gender

In [ ]:
transported_homeplanet = train_data[train_data['Transported'] == 'True']['HomePlanet'].value_counts(normalize=True) * 100
transported_cryosleep = train_data[train_data['Transported'] == 'True']['CryoSleep'].value_counts(normalize=True) * 100
transported_destination = train_data[train_data['Transported'] == 'True']['Destination'].value_counts(normalize=True) * 100
transported_homeplanet, transported_cryosleep, transported_destination

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Drop rows with missing values
train_data_clean = train_data.dropna()

# Encode categorical variables
le = LabelEncoder()
for col in train_data_clean.columns:
    if train_data_clean[col].dtype == 'object':
        train_data_clean[col] = le.fit_transform(train_data_clean[col])

# Split data into features and target
X = train_data_clean.drop('Transported', axis=1)
y = train_data_clean['Transported']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest Classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Get feature importances
importances = clf.feature_importances_
feature_importances = pd.Series(importances, index=X.columns).sort_values(ascending=False)
feature_importances

In [ ]:
test_data = pd.read_csv('test.csv')
sample_submission = pd.read_csv('sample_submission.csv')

# Preprocess test data
test_data_clean = test_data.copy()
for col in test_data_clean.columns:
    if test_data_clean[col].dtype == 'object':
        test_data_clean[col] = le.fit_transform(test_data_clean[col].astype(str))

# Predict 'Transported' for test data
predictions = clf.predict(test_data_clean)

# Format predictions similar to sample_submission
submission = sample_submission.copy()
submission['Transported'] = predictions

# Save predictions as 'predict.csv'
submission.to_csv('predict.csv', index=False)
submission